# Data Preprocessing

In [1]:
import pandas as pd
import swifter
import pickle as pkl

%run "../shared.py"

pd.set_option('display.max_colwidth', 100)

In [2]:
target_dataset = 'tweets.csv'
raw_data_path = f'../dataset/raw/raw_{target_dataset}'
output_path = f'../dataset/processed/processed_{target_dataset}'

In [3]:
df = pd.read_csv(raw_data_path, delimiter='`')

print(df.shape)

(7502, 27)


In [4]:
df = df[['date','content']]

print(df.shape)
df

(7502, 2)


,date,content
0,2022-09-04 23:59:55+00:00,@soeyoto1 @msaid_didu Bukti kalau Pemerintahan Pak @jokowi tidak pro rakyat. Tapi pro oligarki. ...
1,2022-09-04 23:59:38+00:00,"@KompasTV Mentri esdm, trus patokanmu apa? Kantong pertamina udah jebol karna banyak dikorupsi, ..."
2,2022-09-04 23:59:09+00:00,@hnurwahid Ngaku masih subsidi tapi ada swasta yg jua lebih murah? mereka siapa yg subsidi? Pert...
3,2022-09-04 23:58:55+00:00,@Tan_Mar3M Kalau Vivo bisaberani bertahan dengan harga lama bagaimana pula nasib Pertamina ? 🤔
4,2022-09-04 23:57:45+00:00,Menteri BUMN Erick Thohir menegaskan persiapan Indonesia soal transisi energi baru terbarukan. S...
...,...,...
7497,2022-09-03 00:00:54+00:00,"175 Produk UMKM Binaan Pertamina Go Global, Ikut Festival Tong Tong di Belanda https://t.co/LTf7..."
7498,2022-09-03 00:00:36+00:00,"PT Pertamina Patra Niaga, Sub Holding Commercial dan Trading PT Pertamina (Persero) menyebutkan ..."
7499,2022-09-03 00:00:02+00:00,Pemerintah Provinsi Papua Barat sedang mengupayakan dengan mengajak Pertamina agar masalah antre...
7500,2022-09-03 00:00:01+00:00,"terhitung mulai 1 September 2020 pukul 00.00 waktu setempat, Pertamina secara resmi melakukan pe..."


# Section 1: Data cleaning

In [5]:
list_normalize_targets, list_normalize_replacements = load_normalization_list()
list_stopwords = load_stopwords_list()

Normalization list: 1034
Indonesian stopwords: 758
English stopwords: 179
Custom stopwords: 2
Total stopwords: 939


## Section 1.1: Case folding, data cleaning and tokenization data

In [6]:
df['case_folding'] = df['content'].swifter.apply(case_folding)
df['text_clean'] = df['case_folding'].swifter.apply(clean_tweet)
df['tokens'] = df['text_clean'].swifter.apply(tokenize)

print(df.shape)
df

Pandas Apply:   0%|          | 0/7502 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/7502 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/7502 [00:00<?, ?it/s]

(7502, 5)


,date,content,case_folding,text_clean,tokens
0,2022-09-04 23:59:55+00:00,@soeyoto1 @msaid_didu Bukti kalau Pemerintahan Pak @jokowi tidak pro rakyat. Tapi pro oligarki. ...,@soeyoto1 @msaid_didu bukti kalau pemerintahan pak @jokowi tidak pro rakyat. tapi pro oligarki. ...,bukti kalau pemerintahan pak tidak pro rakyat tapi pro oligarki vivo bbmnya lebih murah dari pem...,"[bukti, kalau, pemerintahan, pak, tidak, pro, rakyat, tapi, pro, oligarki, vivo, bbmnya, lebih, ..."
1,2022-09-04 23:59:38+00:00,"@KompasTV Mentri esdm, trus patokanmu apa? Kantong pertamina udah jebol karna banyak dikorupsi, ...","@kompastv mentri esdm, trus patokanmu apa? kantong pertamina udah jebol karna banyak dikorupsi, ...",mentri esdm trus patokanmu apa kantong pertamina udah jebol karna banyak dikorupsi imbasnya raky...,"[mentri, esdm, trus, patokanmu, apa, kantong, pertamina, udah, jebol, karna, banyak, dikorupsi, ..."
2,2022-09-04 23:59:09+00:00,@hnurwahid Ngaku masih subsidi tapi ada swasta yg jua lebih murah? mereka siapa yg subsidi? Pert...,@hnurwahid ngaku masih subsidi tapi ada swasta yg jua lebih murah? mereka siapa yg subsidi? pert...,ngaku masih subsidi tapi ada swasta jua lebih murah mereka siapa subsidi pertamina dipimpin tuka...,"[ngaku, masih, subsidi, tapi, ada, swasta, jua, lebih, murah, mereka, siapa, subsidi, pertamina,..."
3,2022-09-04 23:58:55+00:00,@Tan_Mar3M Kalau Vivo bisaberani bertahan dengan harga lama bagaimana pula nasib Pertamina ? 🤔,@tan_mar3m kalau vivo bisaberani bertahan dengan harga lama bagaimana pula nasib pertamina ? 🤔,kalau vivo bisaberani bertahan dengan harga lama bagaimana pula nasib pertamina,"[kalau, vivo, bisaberani, bertahan, dengan, harga, lama, bagaimana, pula, nasib, pertamina]"
4,2022-09-04 23:57:45+00:00,Menteri BUMN Erick Thohir menegaskan persiapan Indonesia soal transisi energi baru terbarukan. S...,menteri bumn erick thohir menegaskan persiapan indonesia soal transisi energi baru terbarukan. s...,menteri bumn erick thohir menegaskan persiapan indonesia soal transisi energi baru terbarukan sa...,"[menteri, bumn, erick, thohir, menegaskan, persiapan, indonesia, soal, transisi, energi, baru, t..."
...,...,...,...,...,...
7497,2022-09-03 00:00:54+00:00,"175 Produk UMKM Binaan Pertamina Go Global, Ikut Festival Tong Tong di Belanda https://t.co/LTf7...","175 produk umkm binaan pertamina go global, ikut festival tong tong di belanda https://t.co/ltf7...",produk umkm binaan pertamina global ikut festival tong tong belanda,"[produk, umkm, binaan, pertamina, global, ikut, festival, tong, tong, belanda]"
7498,2022-09-03 00:00:36+00:00,"PT Pertamina Patra Niaga, Sub Holding Commercial dan Trading PT Pertamina (Persero) menyebutkan ...","pt pertamina patra niaga, sub holding commercial dan trading pt pertamina (persero) menyebutkan ...",pertamina patra niaga sub holding commercial dan trading pertamina persero menyebutkan telah mel...,"[pertamina, patra, niaga, sub, holding, commercial, dan, trading, pertamina, persero, menyebutka..."
7499,2022-09-03 00:00:02+00:00,Pemerintah Provinsi Papua Barat sedang mengupayakan dengan mengajak Pertamina agar masalah antre...,pemerintah provinsi papua barat sedang mengupayakan dengan mengajak pertamina agar masalah antre...,pemerintah provinsi papua barat sedang mengupayakan dengan mengajak pertamina agar masalah antre...,"[pemerintah, provinsi, papua, barat, sedang, mengupayakan, dengan, mengajak, pertamina, agar, ma..."
7500,2022-09-03 00:00:01+00:00,"terhitung mulai 1 September 2020 pukul 00.00 waktu setempat, Pertamina secara resmi melakukan pe...","terhitung mulai 1 september 2020 pukul 00.00 waktu setempat, pertamina secara resmi melakukan pe...",terhitung mulai september pukul waktu setempat pertamina secara resmi melakukan penyesuaian beru...,"[terhitung, mulai, september, pukul, waktu, setempat, pertamina, secara, resmi, melakukan, penye..."


## Section 1.2: Normalization and Stopwords removal

In [7]:
df['tokens'] = df['tokens'].swifter.apply(normalize, args=(list_normalize_targets, list_normalize_replacements))
df['tokens'] = df['tokens'].swifter.apply(remove_stopwords, args=(list_stopwords,))

print(df.shape)
df

Pandas Apply:   0%|          | 0/7502 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/7502 [00:00<?, ?it/s]

(7502, 5)


,date,content,case_folding,text_clean,tokens
0,2022-09-04 23:59:55+00:00,@soeyoto1 @msaid_didu Bukti kalau Pemerintahan Pak @jokowi tidak pro rakyat. Tapi pro oligarki. ...,@soeyoto1 @msaid_didu bukti kalau pemerintahan pak @jokowi tidak pro rakyat. tapi pro oligarki. ...,bukti kalau pemerintahan pak tidak pro rakyat tapi pro oligarki vivo bbmnya lebih murah dari pem...,"[bukti, pemerintahan, pro, rakyat, pro, oligarki, bbmnya, murah, pemerintah, harga, kebodohan]"
1,2022-09-04 23:59:38+00:00,"@KompasTV Mentri esdm, trus patokanmu apa? Kantong pertamina udah jebol karna banyak dikorupsi, ...","@kompastv mentri esdm, trus patokanmu apa? kantong pertamina udah jebol karna banyak dikorupsi, ...",mentri esdm trus patokanmu apa kantong pertamina udah jebol karna banyak dikorupsi imbasnya raky...,"[menteri, esdm, patokanmu, kantong, jebol, dikorupsi, imbasnya, rakyat, menanggung]"
2,2022-09-04 23:59:09+00:00,@hnurwahid Ngaku masih subsidi tapi ada swasta yg jua lebih murah? mereka siapa yg subsidi? Pert...,@hnurwahid ngaku masih subsidi tapi ada swasta yg jua lebih murah? mereka siapa yg subsidi? pert...,ngaku masih subsidi tapi ada swasta jua lebih murah mereka siapa subsidi pertamina dipimpin tuka...,"[mengaku, subsidi, swasta, jual, murah, subsidi, pimpin, tukang, memaki, nasib, rakyatnya, gitu,..."
3,2022-09-04 23:58:55+00:00,@Tan_Mar3M Kalau Vivo bisaberani bertahan dengan harga lama bagaimana pula nasib Pertamina ? 🤔,@tan_mar3m kalau vivo bisaberani bertahan dengan harga lama bagaimana pula nasib pertamina ? 🤔,kalau vivo bisaberani bertahan dengan harga lama bagaimana pula nasib pertamina,"[berani, bertahan, harga, nasib]"
4,2022-09-04 23:57:45+00:00,Menteri BUMN Erick Thohir menegaskan persiapan Indonesia soal transisi energi baru terbarukan. S...,menteri bumn erick thohir menegaskan persiapan indonesia soal transisi energi baru terbarukan. s...,menteri bumn erick thohir menegaskan persiapan indonesia soal transisi energi baru terbarukan sa...,"[menteri, bumn, erick, thohir, persiapan, indonesia, transisi, energi, terbarukan, salah, upayan..."
...,...,...,...,...,...
7497,2022-09-03 00:00:54+00:00,"175 Produk UMKM Binaan Pertamina Go Global, Ikut Festival Tong Tong di Belanda https://t.co/LTf7...","175 produk umkm binaan pertamina go global, ikut festival tong tong di belanda https://t.co/ltf7...",produk umkm binaan pertamina global ikut festival tong tong belanda,"[produk, umkm, binaan, global, festival, tong, tong, belanda]"
7498,2022-09-03 00:00:36+00:00,"PT Pertamina Patra Niaga, Sub Holding Commercial dan Trading PT Pertamina (Persero) menyebutkan ...","pt pertamina patra niaga, sub holding commercial dan trading pt pertamina (persero) menyebutkan ...",pertamina patra niaga sub holding commercial dan trading pertamina persero menyebutkan telah mel...,"[patra, niaga, sub, holding, commercial, trading, persero, penyesuaian, harga, bbm, non, subsidi..."
7499,2022-09-03 00:00:02+00:00,Pemerintah Provinsi Papua Barat sedang mengupayakan dengan mengajak Pertamina agar masalah antre...,pemerintah provinsi papua barat sedang mengupayakan dengan mengajak pertamina agar masalah antre...,pemerintah provinsi papua barat sedang mengupayakan dengan mengajak pertamina agar masalah antre...,"[pemerintah, provinsi, papua, barat, mengupayakan, mengajak, antrian, pengisian, bbm, spbu, berd..."
7500,2022-09-03 00:00:01+00:00,"terhitung mulai 1 September 2020 pukul 00.00 waktu setempat, Pertamina secara resmi melakukan pe...","terhitung mulai 1 september 2020 pukul 00.00 waktu setempat, pertamina secara resmi melakukan pe...",terhitung mulai september pukul waktu setempat pertamina secara resmi melakukan penyesuaian beru...,"[terhitung, september, resmi, penyesuaian, penurunan, harga, produk, pertamax, turbo, ron, dexli..."


## Section 1.3: Remove Data

In [8]:
def remove_data(df, key_name = 'tokens'):
	print(f"Before: {df.shape}")

	# Remove duplicate data
	df.drop_duplicates(subset=key_name, keep="first", inplace=True)

	# Remove empty data
	df.dropna(subset=[key_name], inplace=True)

	# Remove data with length less than 2
	df = df[df[key_name].map(len) > 2]
	df.reset_index(inplace=True)

	print(f"After: {df.shape}")
	return df

# df = remove_data(df)
# df

# Section 1: Data Stemming

In [9]:
terms_dict = stemming(df['tokens'])

Unique terms: 12074
On processing... 0 terms have been stemmed
On processing... 1000 terms have been stemmed
On processing... 2000 terms have been stemmed
On processing... 3000 terms have been stemmed
On processing... 4000 terms have been stemmed
On processing... 5000 terms have been stemmed
On processing... 6000 terms have been stemmed
On processing... 7000 terms have been stemmed
On processing... 8000 terms have been stemmed
On processing... 9000 terms have been stemmed
On processing... 10000 terms have been stemmed
On processing... 11000 terms have been stemmed
On processing... 12000 terms have been stemmed


In [10]:
with open('../dataset/wordlist/stemming_terms.pkl', "wb") as f:
	pkl.dump(terms_dict, f)

In [11]:
def apply_stemmed_tokens(tokens):
	return [terms_dict[token] for token in tokens]

df['tokens_ready'] = df['tokens'].swifter.apply(apply_stemmed_tokens)

print(df.shape)
df

Pandas Apply:   0%|          | 0/7502 [00:00<?, ?it/s]

(7502, 6)


,date,content,case_folding,text_clean,tokens,tokens_ready
0,2022-09-04 23:59:55+00:00,@soeyoto1 @msaid_didu Bukti kalau Pemerintahan Pak @jokowi tidak pro rakyat. Tapi pro oligarki. ...,@soeyoto1 @msaid_didu bukti kalau pemerintahan pak @jokowi tidak pro rakyat. tapi pro oligarki. ...,bukti kalau pemerintahan pak tidak pro rakyat tapi pro oligarki vivo bbmnya lebih murah dari pem...,"[bukti, pemerintahan, pro, rakyat, pro, oligarki, bbmnya, murah, pemerintah, harga, kebodohan]","[bukti, perintah, pro, rakyat, pro, oligarki, bbmnya, murah, perintah, harga, bodoh]"
1,2022-09-04 23:59:38+00:00,"@KompasTV Mentri esdm, trus patokanmu apa? Kantong pertamina udah jebol karna banyak dikorupsi, ...","@kompastv mentri esdm, trus patokanmu apa? kantong pertamina udah jebol karna banyak dikorupsi, ...",mentri esdm trus patokanmu apa kantong pertamina udah jebol karna banyak dikorupsi imbasnya raky...,"[menteri, esdm, patokanmu, kantong, jebol, dikorupsi, imbasnya, rakyat, menanggung]","[menteri, esdm, patok, kantong, jebol, korupsi, imbas, rakyat, tanggung]"
2,2022-09-04 23:59:09+00:00,@hnurwahid Ngaku masih subsidi tapi ada swasta yg jua lebih murah? mereka siapa yg subsidi? Pert...,@hnurwahid ngaku masih subsidi tapi ada swasta yg jua lebih murah? mereka siapa yg subsidi? pert...,ngaku masih subsidi tapi ada swasta jua lebih murah mereka siapa subsidi pertamina dipimpin tuka...,"[mengaku, subsidi, swasta, jual, murah, subsidi, pimpin, tukang, memaki, nasib, rakyatnya, gitu,...","[aku, subsidi, swasta, jual, murah, subsidi, pimpin, tukang, maki, nasib, rakyat, gitu, puja, pu..."
3,2022-09-04 23:58:55+00:00,@Tan_Mar3M Kalau Vivo bisaberani bertahan dengan harga lama bagaimana pula nasib Pertamina ? 🤔,@tan_mar3m kalau vivo bisaberani bertahan dengan harga lama bagaimana pula nasib pertamina ? 🤔,kalau vivo bisaberani bertahan dengan harga lama bagaimana pula nasib pertamina,"[berani, bertahan, harga, nasib]","[berani, tahan, harga, nasib]"
4,2022-09-04 23:57:45+00:00,Menteri BUMN Erick Thohir menegaskan persiapan Indonesia soal transisi energi baru terbarukan. S...,menteri bumn erick thohir menegaskan persiapan indonesia soal transisi energi baru terbarukan. s...,menteri bumn erick thohir menegaskan persiapan indonesia soal transisi energi baru terbarukan sa...,"[menteri, bumn, erick, thohir, persiapan, indonesia, transisi, energi, terbarukan, salah, upayan...","[menteri, bumn, erick, thohir, siap, indonesia, transisi, energi, baru, salah, upaya, kerja]"
...,...,...,...,...,...,...
7497,2022-09-03 00:00:54+00:00,"175 Produk UMKM Binaan Pertamina Go Global, Ikut Festival Tong Tong di Belanda https://t.co/LTf7...","175 produk umkm binaan pertamina go global, ikut festival tong tong di belanda https://t.co/ltf7...",produk umkm binaan pertamina global ikut festival tong tong belanda,"[produk, umkm, binaan, global, festival, tong, tong, belanda]","[produk, umkm, bina, global, festival, tong, tong, belanda]"
7498,2022-09-03 00:00:36+00:00,"PT Pertamina Patra Niaga, Sub Holding Commercial dan Trading PT Pertamina (Persero) menyebutkan ...","pt pertamina patra niaga, sub holding commercial dan trading pt pertamina (persero) menyebutkan ...",pertamina patra niaga sub holding commercial dan trading pertamina persero menyebutkan telah mel...,"[patra, niaga, sub, holding, commercial, trading, persero, penyesuaian, harga, bbm, non, subsidi...","[patra, niaga, sub, holding, commercial, trading, persero, sesuai, harga, bbm, non, subsidi, dex..."
7499,2022-09-03 00:00:02+00:00,Pemerintah Provinsi Papua Barat sedang mengupayakan dengan mengajak Pertamina agar masalah antre...,pemerintah provinsi papua barat sedang mengupayakan dengan mengajak pertamina agar masalah antre...,pemerintah provinsi papua barat sedang mengupayakan dengan mengajak pertamina agar masalah antre...,"[pemerintah, provinsi, papua, barat, mengupayakan, mengajak, antrian, pengisian, bbm, spbu, berd...","[perintah, provinsi, papua, barat, upaya, ajak, antri, isi, bbm, spbu, dampak, distri

In [13]:
def combine_tokens(tokens):
	return ' '.join(tokens)

df['review'] = df['tokens_ready'].swifter.apply(combine_tokens)
pd.set_option('display.max_colwidth', 50)
df

Pandas Apply:   0%|          | 0/7502 [00:00<?, ?it/s]

,date,content,case_folding,text_clean,tokens,tokens_ready,review
0,2022-09-04 23:59:55+00:00,@soeyoto1 @msaid_didu Bukti kalau Pemerintahan...,@soeyoto1 @msaid_didu bukti kalau pemerintahan...,bukti kalau pemerintahan pak tidak pro rakyat ...,"[bukti, pemerintahan, pro, rakyat, pro, oligar...","[bukti, perintah, pro, rakyat, pro, oligarki, ...",bukti perintah pro rakyat pro oligarki bbmnya ...
1,2022-09-04 23:59:38+00:00,"@KompasTV Mentri esdm, trus patokanmu apa? Kan...","@kompastv mentri esdm, trus patokanmu apa? kan...",mentri esdm trus patokanmu apa kantong pertami...,"[menteri, esdm, patokanmu, kantong, jebol, dik...","[menteri, esdm, patok, kantong, jebol, korupsi...",menteri esdm patok kantong jebol korupsi imbas...
2,2022-09-04 23:59:09+00:00,@hnurwahid Ngaku masih subsidi tapi ada swasta...,@hnurwahid ngaku masih subsidi tapi ada swasta...,ngaku masih subsidi tapi ada swasta jua lebih ...,"[mengaku, subsidi, swasta, jual, murah, subsid...","[aku, subsidi, swasta, jual, murah, subsidi, p...",aku subsidi swasta jual murah subsidi pimpin t...
3,2022-09-04 23:58:55+00:00,@Tan_Mar3M Kalau Vivo bisaberani bertahan deng...,@tan_mar3m kalau vivo bisaberani bertahan deng...,kalau vivo bisaberani bertahan dengan harga la...,"[berani, bertahan, harga, nasib]","[berani, tahan, harga, nasib]",berani tahan harga nasib
4,2022-09-04 23:57:45+00:00,Menteri BUMN Erick Thohir menegaskan persiapan...,menteri bumn erick thohir menegaskan persiapan...,menteri bumn erick thohir menegaskan persiapan...,"[menteri, bumn, erick, thohir, persiapan, indo...","[menteri, bumn, erick, thohir, siap, indonesia...",menteri bumn erick thohir siap indonesia trans...
...,...,...,...,...,...,...,...
7497,2022-09-03 00:00:54+00:00,"175 Produk UMKM Binaan Pertamina Go Global, Ik...","175 produk umkm binaan pertamina go global, ik...",produk umkm binaan pertamina global ikut festi...,"[produk, umkm, binaan, global, festival, tong,...","[produk, umkm, bina, global, festival, tong, t...",produk umkm bina global festival tong tong bel...
7498,2022-09-03 00:00:36+00:00,"PT Pertamina Patra Niaga, Sub Holding Commerci...","pt pertamina patra niaga, sub holding commerci...",pertamina patra niaga sub holding commercial d...,"[patra, niaga, sub, holding, commercial, tradi...","[patra, niaga, sub, holding, commercial, tradi...",patra niaga sub holding commercial trading per...
7499,2022-09-03 00:00:02+00:00,Pemerintah Provinsi Papua Barat sedang mengupa...,pemerintah provinsi papua barat sedang mengupa...,pemerintah provinsi papua barat sedang mengupa...,"[pemerintah, provinsi, papua, barat, mengupaya...","[perintah, provinsi, papua, barat, upaya, ajak...",perintah provinsi papua barat upaya ajak antri...
7500,2022-09-03 00:00:01+00:00,terhitung mulai 1 September 2020 pukul 00.00 w...,terhitung mulai 1 september 2020 pukul 00.00 w...,terhitung mulai september pukul waktu setempat...,"[terhitung, september, resmi, penyesuaian, pen...","[hitung, september, resmi, sesuai, turun, harg...",hitung september resmi sesuai turun harga prod...


In [17]:
duplicate_data = df[df.duplicated(subset='review')].sort_values(by='review')

duplicate_data

,date,content,case_folding,text_clean,tokens,tokens_ready,review
3435,2022-09-04 03:29:31+00:00,@reza_doni @Megatop99 @jokowi @pertamina Ini a...,@reza_doni @megatop99 @jokowi @pertamina ini a...,ini ada lagi,[],[],
843,2022-09-04 13:58:10+00:00,@mwicaza dia juga yang pertamina,@mwicaza dia juga yang pertamina,dia juga yang pertamina,[],[],
3925,2022-09-04 01:11:46+00:00,Benarkah.. @pertamina @DPR_RI @FPKSDPRRI @mohm...,benarkah.. @pertamina @dpr_ri @fpksdprri @mohm...,benarkah,[],[],
3745,2022-09-04 02:06:11+00:00,@Megatop99 @menda_endy @jokowi @pertamina Mema...,@megatop99 @menda_endy @jokowi @pertamina mema...,memang seperti itu,[],[],
3542,2022-09-04 03:00:45+00:00,@worksfess pertamina lah ya,@worksfess pertamina lah ya,pertamina lah,[],[],
...,...,...,...,...,...,...,...
7274,2022-09-03 05:46:14+00:00,"Wuihh Ikut Festival Tong Tong di Belanda, 175 ...","wuihh ikut festival tong tong di belanda, 175 ...",wuihh ikut festival tong tong belanda produk t...,"[wuihh, festival, tong, tong, belanda, produk,...","[wuihh, festival, tong, tong, belanda, produk,...",wuihh festival tong tong belanda produk baik u...
6113,2022-09-03 11:15:02+00:00,"Wuihh Ikut Festival Tong Tong di Belanda, 175 ...","wuihh ikut festival tong tong di belanda, 175 ...",wuihh ikut festival tong tong belanda produk t...,"[wuihh, festival, tong, tong, belanda, produk,...","[wuihh, festival, tong, tong, belanda, produk,...",wuihh festival tong tong belanda produk baik u...
6864,2022-09-03 08:21:09+00:00,"Yeay Ikut Festival Tong Tong di Belanda, 175 P...","yeay ikut festival tong tong di belanda, 175 p...",yeay ikut festival tong tong belanda produk te...,"[yeay, festival, tong, tong, belanda, produk, ...","[yeay, festival, tong, tong, belanda, produk, ...",yeay festival tong tong belanda produk baik um...
1151,2022-09-04 12:45:56+00:00,"Yeayy Pastikan Pasokan BBM Aman, Dirut Pertami...","yeayy pastikan pasokan bbm aman, dirut pertami...",yeayy pastikan pasokan bbm aman dirut pertamin...,"[yeayy, pastikan, pasokan, bbm, aman, direktur...","[yeayy, pasti, pasok, bbm, aman, direktur utam...",yeayy pasti pasok bbm aman direktur utama awas...


In [19]:
df['tokens_ready'] = df['tokens_ready'].swifter.apply(remove_stopwords, args=(list_stopwords,))
df['review'] = df['tokens_ready'].swifter.apply(combine_tokens)

df = remove_data(df, 'review')

print(df.shape)
df.head()

Pandas Apply:   0%|          | 0/7080 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/7080 [00:00<?, ?it/s]

Before: (7080, 7)
After: (7079, 8)
(7079, 8)


,index,date,content,case_folding,text_clean,tokens,tokens_ready,review
0,0,2022-09-04 23:59:55+00:00,@soeyoto1 @msaid_didu Bukti kalau Pemerintahan...,@soeyoto1 @msaid_didu bukti kalau pemerintahan...,bukti kalau pemerintahan pak tidak pro rakyat ...,"[bukti, pemerintahan, pro, rakyat, pro, oligar...","[bukti, perintah, pro, rakyat, pro, oligarki, ...",bukti perintah pro rakyat pro oligarki bbmnya ...
1,1,2022-09-04 23:59:38+00:00,"@KompasTV Mentri esdm, trus patokanmu apa? Kan...","@kompastv mentri esdm, trus patokanmu apa? kan...",mentri esdm trus patokanmu apa kantong pertami...,"[menteri, esdm, patokanmu, kantong, jebol, dik...","[menteri, esdm, patok, kantong, jebol, korupsi...",menteri esdm patok kantong jebol korupsi imbas...
2,2,2022-09-04 23:59:09+00:00,@hnurwahid Ngaku masih subsidi tapi ada swasta...,@hnurwahid ngaku masih subsidi tapi ada swasta...,ngaku masih subsidi tapi ada swasta jua lebih ...,"[mengaku, subsidi, swasta, jual, murah, subsid...","[subsidi, swasta, jual, murah, subsidi, pimpin...",subsidi swasta jual murah subsidi pimpin tukan...
3,3,2022-09-04 23:58:55+00:00,@Tan_Mar3M Kalau Vivo bisaberani bertahan deng...,@tan_mar3m kalau vivo bisaberani bertahan deng...,kalau vivo bisaberani bertahan dengan harga la...,"[berani, bertahan, harga, nasib]","[berani, tahan, harga, nasib]",berani tahan harga nasib
4,4,2022-09-04 23:57:45+00:00,Menteri BUMN Erick Thohir menegaskan persiapan...,menteri bumn erick thohir menegaskan persiapan...,menteri bumn erick thohir menegaskan persiapan...,"[menteri, bumn, erick, thohir, persiapan, indo...","[menteri, bumn, erick, thohir, indonesia, tran...",menteri bumn erick thohir indonesia transisi e...


In [21]:
df

,index,date,content,case_folding,text_clean,tokens,tokens_ready,review
0,0,2022-09-04 23:59:55+00:00,@soeyoto1 @msaid_didu Bukti kalau Pemerintahan...,@soeyoto1 @msaid_didu bukti kalau pemerintahan...,bukti kalau pemerintahan pak tidak pro rakyat ...,"[bukti, pemerintahan, pro, rakyat, pro, oligar...","[bukti, perintah, pro, rakyat, pro, oligarki, ...",bukti perintah pro rakyat pro oligarki bbmnya ...
1,1,2022-09-04 23:59:38+00:00,"@KompasTV Mentri esdm, trus patokanmu apa? Kan...","@kompastv mentri esdm, trus patokanmu apa? kan...",mentri esdm trus patokanmu apa kantong pertami...,"[menteri, esdm, patokanmu, kantong, jebol, dik...","[menteri, esdm, patok, kantong, jebol, korupsi...",menteri esdm patok kantong jebol korupsi imbas...
2,2,2022-09-04 23:59:09+00:00,@hnurwahid Ngaku masih subsidi tapi ada swasta...,@hnurwahid ngaku masih subsidi tapi ada swasta...,ngaku masih subsidi tapi ada swasta jua lebih ...,"[mengaku, subsidi, swasta, jual, murah, subsid...","[subsidi, swasta, jual, murah, subsidi, pimpin...",subsidi swasta jual murah subsidi pimpin tukan...
3,3,2022-09-04 23:58:55+00:00,@Tan_Mar3M Kalau Vivo bisaberani bertahan deng...,@tan_mar3m kalau vivo bisaberani bertahan deng...,kalau vivo bisaberani bertahan dengan harga la...,"[berani, bertahan, harga, nasib]","[berani, tahan, harga, nasib]",berani tahan harga nasib
4,4,2022-09-04 23:57:45+00:00,Menteri BUMN Erick Thohir menegaskan persiapan...,menteri bumn erick thohir menegaskan persiapan...,menteri bumn erick thohir menegaskan persiapan...,"[menteri, bumn, erick, thohir, persiapan, indo...","[menteri, bumn, erick, thohir, indonesia, tran...",menteri bumn erick thohir indonesia transisi e...
...,...,...,...,...,...,...,...,...
7074,7497,2022-09-03 00:00:54+00:00,"175 Produk UMKM Binaan Pertamina Go Global, Ik...","175 produk umkm binaan pertamina go global, ik...",produk umkm binaan pertamina global ikut festi...,"[produk, umkm, binaan, global, festival, tong,...","[produk, umkm, bina, global, festival, tong, t...",produk umkm bina global festival tong tong bel...
7075,7498,2022-09-03 00:00:36+00:00,"PT Pertamina Patra Niaga, Sub Holding Commerci...","pt pertamina patra niaga, sub holding commerci...",pertamina patra niaga sub holding commercial d...,"[patra, niaga, sub, holding, commercial, tradi...","[patra, niaga, sub, holding, commercial, tradi...",patra niaga sub holding commercial trading per...
7076,7499,2022-09-03 00:00:02+00:00,Pemerintah Provinsi Papua Barat sedang mengupa...,pemerintah provinsi papua barat sedang mengupa...,pemerintah provinsi papua barat sedang mengupa...,"[pemerintah, provinsi, papua, barat, mengupaya...","[perintah, provinsi, papua, barat, upaya, ajak...",perintah provinsi papua barat upaya ajak antri...
7077,7500,2022-09-03 00:00:01+00:00,terhitung mulai 1 September 2020 pukul 00.00 w...,terhitung mulai 1 september 2020 pukul 00.00 w...,terhitung mulai september pukul waktu setempat...,"[terhitung, september, resmi, penyesuaian, pen...","[hitung, september, resmi, sesuai, turun, harg...",hitung september resmi sesuai turun harga prod...


In [20]:
df.to_csv(output_path, index=False, header=True, sep=';')